In [1]:
import os
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.translate import bleu_score

from euskalToken import EuskalToken

import warnings
warnings.filterwarnings("ignore")

def avg_of_sentence_bleu_scores(groundtruth_filename, prediction_filename, language):
    
    list_of_scores = []
    
    if language == "euskara":
        tokenize = EuskalToken().tokenize
    elif language == "english":
        tokenize = word_tokenize
            
    with open(groundtruth_filename, encoding="utf8") as gtfile, open(prediction_filename, encoding="utf8") as predfile:
   
        for i, (gt_sentence, predicted_sentence) in enumerate(zip(gtfile, predfile)):   
        
            gt_list_of_tokens = tokenize(gt_sentence)
            predicted_list_of_tokens = predicted_sentence.split()

            list_of_scores.append(bleu_score.sentence_bleu([gt_list_of_tokens], predicted_list_of_tokens))
            
        array_of_scores = np.asarray(list_of_scores)
        avg_bleu_scores = array_of_scores.mean()
          
    return avg_bleu_scores


/home/orlandom/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/orlandom/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/orlandom/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
groundtruth_filename = "./evaldata/berriak.en"
prediction_filename = "./predictions/predicted_berriak.unsup.en"

# Without smoothing
score = avg_of_sentence_bleu_scores(groundtruth_filename, prediction_filename, "english")
print("Non smoothed BLEU score: {:.2%}".format(score))

Non smoothed BLEU score: 0.14%


In [3]:
groundtruth_filename = "./evaldata/berriak.en"
prediction_filename = "./predictions/predicted_berriak.semisup.en"

# Without smoothing
score = avg_of_sentence_bleu_scores(groundtruth_filename, prediction_filename, "english")
print("Non smoothed BLEU score: {:.2%}".format(score))

Non smoothed BLEU score: 0.48%


In [4]:
groundtruth_filename = "./evaldata/berriak.eu"
prediction_filename = "./predictions/predicted_berriak.unsup.eu"

# Without smoothing
score = avg_of_sentence_bleu_scores(groundtruth_filename, prediction_filename, "euskara")
print("Non smoothed BLEU score: {:.2%}".format(score))

Non smoothed BLEU score: 0.03%


In [5]:
groundtruth_filename = "./evaldata/berriak.eu"
prediction_filename = "./predictions/predicted_berriak.semisup.eu"

# Without smoothing
score = avg_of_sentence_bleu_scores(groundtruth_filename, prediction_filename, "euskara")
print("Non smoothed BLEU score: {:.2%}".format(score))

Non smoothed BLEU score: 0.17%
